# Pretrained Neural Network Classifier w/o feature reduction
See [this](https://www.kaggle.com/andreasthomasen/pytorch-nn-model) for implementation details. Here we are running a pretrained version of that model.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import torch
import torch.nn as nn
import torch.optim as optim

if torch.cuda.is_available():
    dev = torch.device("cuda")
else:
    dev = torch.device("cpu")

import pickle
    
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# Defining and importing the model

In [ ]:
e_size = 64
fc_input = 130
h_dims = [512,512,256,128]
dropout_rate = 0.5
epochs = 2000
minibatch_size = 100000

class MarketPredictor(nn.Module):
    def __init__(self):
        super(MarketPredictor, self).__init__()
        
        self.deep = nn.Sequential(
            nn.Linear(fc_input,h_dims[0]),
            nn.BatchNorm1d(h_dims[0]),
            nn.LeakyReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(h_dims[0],h_dims[1]),
            nn.BatchNorm1d(h_dims[1]),
            nn.LeakyReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(h_dims[1],h_dims[2]),
            nn.BatchNorm1d(h_dims[2]),
            nn.LeakyReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(h_dims[2],h_dims[3]),
            nn.BatchNorm1d(h_dims[3]),
            nn.LeakyReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(h_dims[3],e_size),
            nn.BatchNorm1d(e_size),
            nn.LeakyReLU(),
            nn.Dropout(dropout_rate)
            )
        self.reduce = nn.utils.weight_norm(nn.Linear(e_size,5))
        
    def forward(self,xf):
        f_out = self.deep(xf)
        ef_out = self.reduce(f_out)
        
        return ef_out
        

In [ ]:
path = '/kaggle/input/pytorch-nn-model-w-o-feature-reduction/marketpredictor_state_dict_'+str(epochs)+'epochs.pt'
model = MarketPredictor()
model.load_state_dict(torch.load(path,map_location=dev))
model.to(dev)
model.eval()

In [ ]:
with open('/kaggle/input/pytorch-nn-model-w-o-feature-reduction/feature_processing.pkl', 'rb') as f:
    sc, maxindex, fill_val = pickle.load(f)

In [ ]:
feature_names = ['feature_'+str(i) for i in range(130)]
exclude = np.where([maxindex[i,1] > 100 and maxindex [i,2] > 1 for i in range(129) ])[0]

In [ ]:
import janestreet
env = janestreet.make_env()
iter_test = env.iter_test()

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    if test_df['weight'].item() == 0:
        sample_prediction_df.action = 0
    else:
        test_df_features = test_df[feature_names].to_numpy()
        for i in exclude:
            if test_df_features[0,i+1] == maxindex[i,0]:
                test_df_features[0,i+1] = fill_val[i]
        
        nans = np.isnan(test_df_features)
        for i in range(130):
            if nans[0,i]:
                test_df_features[0,i] = fill_val[i]
        
        test_df_features = sc.transform(test_df_features)
        ftensor = torch.tensor(test_df_features,dtype=torch.float,device=dev)
        s = model(ftensor)[0,0].item()
        sample_prediction_df.action = int(np.round(1/(1+np.exp(-s))))
        
    env.predict(sample_prediction_df)